# K-means 1D - Implementação Sequencial


### Guilherme Pereira Campos - 163787 

---

Este notebook tem a implementação da **etapa 0: versão sequencial** do K-means. 

 O que será feito: 
1. **Gerar os dados** de forma pseudoaleatória e reprodutiva em um formato com 1 coluna e sem cabeçalho. 
2. **Visualizar** os dados gerados.
3. **Implementar o código** C `kmeans_Id_naive.c` para a versão sequencial. 
4. **Executar** o código em C e coletar **SSE por iteração, tempo total, iterações.** que servirão de baseline. 
5. **Analisar** graficamente a convergência do algoritmo e as outras métricas. 

### 1. Geração dos dados pseudoaleatórios e reprodutivos

Será usado a biblioteca `NumPy` para gerar dados sintéticos 1D que serão usados posteriormente no K-means. O processo garante a reprodutibilidade. 

A geração consiste em: 
1. Fixar a semente aleatória `np.random.seed(24)`para garantir reprodutibilidade.
2. Criar pontos distribuídos em faixas distintas (ex: 0, 10, 20, 30) usando distribuição normal.
3. Gerar três conjuntos de teste com tamanhos crescentes:
	- Pequeno: N=10⁴ pontos, K=4 clusters
	- Médio: N=10⁵ pontos, K=8 clusters
	- Grande: N=10⁶ pontos, K=16 clusters

4. Selecionar centróides iniciais aleatoriamente dos dados gerados para cada conjunto.
5. Embaralhar os pontos usando `np.random.shuffle`
6. Converter os dados para DataFrame do pandas e salvar em arquivos CSV (1 coluna, sem cabeçalho, sem índice) usando `to_csv(index=False, header=False)`:
	- `dados_*.csv` contendo os N pontos
	- `centroides_*.csv` contendo os K centróides iniciais

In [7]:
# Importar as bibliotecas necessárias
import numpy as np
import pandas as pd
import os

# Fixar semente para reprodutibilidade
np.random.seed(24)

def gerar_dados_clusters(N, K, faixas, desvio=2.0, nome_base=''):
    """
    Gera dados 1D com K clusters em faixas distintas
    """
    pontos_por_cluster = N // K
    pontos = []
    
    print(f"\n{'='*60}")
    print(f"Gerando conjunto: {nome_base.upper()}")
    print(f"{'='*60}")
    print(f"N = {N:,} pontos")
    print(f"K = {K} clusters")
    print(f"Centros reais dos clusters: {faixas}")
    print(f"Desvio padrão: {desvio}")
    
    # Gerar pontos ao redor de cada centro
    for i, centro in enumerate(faixas):
        pontos_cluster = np.random.normal(centro, desvio, pontos_por_cluster)
        pontos.extend(pontos_cluster)
        print(f"  Cluster {i}: ~{pontos_por_cluster:,} pontos ao redor de {centro}")
    
    # Adicionar pontos restantes (se N não for divisível por K)
    pontos_restantes = N - len(pontos)
    if pontos_restantes > 0:
        centro_extra = np.random.choice(faixas)
        pontos_extras = np.random.normal(centro_extra, desvio, pontos_restantes)
        pontos.extend(pontos_extras)
    
    # Converter para array e embaralhar
    pontos = np.array(pontos)
    np.random.shuffle(pontos)
    
    # Gerar centróides iniciais (escolher K pontos aleatórios dos dados)
    indices = np.random.choice(len(pontos), K, replace=False)
    centroides_iniciais = pontos[indices]
    
    print(f"\nCentróides iniciais escolhidos: {centroides_iniciais}")
    
    # Converter para DataFrame do pandas
    df_dados = pd.DataFrame(pontos)
    df_centroides = pd.DataFrame(centroides_iniciais)
    
    # Salvar em CSV (sem índice, sem cabeçalho) - usando ../ para subir um nível
    arquivo_dados = f'../dados/dados_{nome_base}.csv'
    arquivo_centroides = f'../dados/centroides_{nome_base}.csv'
    
    df_dados.to_csv(arquivo_dados, index=False, header=False)
    df_centroides.to_csv(arquivo_centroides, index=False, header=False)
    
    print(f"\n✓ Arquivos salvos:")
    print(f"  - {arquivo_dados}")
    print(f"  - {arquivo_centroides}")
    
    return pontos, centroides_iniciais, faixas

# Conjunto pequeno: N=10^4, K=4
faixas_pequeno = [0, 10, 20, 30]
N_pequeno = 10**4
K_pequeno = 4

dados_pequeno, cent_pequeno, faixas_p = gerar_dados_clusters(
    N=N_pequeno,
    K=K_pequeno,
    faixas=faixas_pequeno,
    desvio=2.0,
    nome_base='pequeno'
)

# Conjunto médio: N=10^5, K=8
faixas_medio = [0, 10, 20, 30, 40, 50, 60, 70]
N_medio = 10**5
K_medio = 8

dados_medio, cent_medio, faixas_m = gerar_dados_clusters(
    N=N_medio,
    K=K_medio,
    faixas=faixas_medio,
    desvio=2.5,
    nome_base='medio'
)

# Conjunto grande: N=10^6, K=16
faixas_grande = [i*10 for i in range(16)]  
N_grande = 10**6
K_grande = 16

dados_grande, cent_grande, faixas_g = gerar_dados_clusters(
    N=N_grande,
    K=K_grande,
    faixas=faixas_grande,
    desvio=3.0,
    nome_base='grande'
)

print(f"\n{'='*60}")
print("Resumo dos dados gerados")
print(f"{'='*60}")
print(f"Pequeno: {N_pequeno:,} pontos, {K_pequeno} clusters")
print(f"Médio: {N_medio:,} pontos, {K_medio} clusters")
print(f"Grande: {N_grande:,} pontos, {K_grande} clusters")
print(f"\nTodos os arquivos foram salvos na pasta '../dados/'")



Gerando conjunto: PEQUENO
N = 10,000 pontos
K = 4 clusters
Centros reais dos clusters: [0, 10, 20, 30]
Desvio padrão: 2.0
  Cluster 0: ~2,500 pontos ao redor de 0
  Cluster 1: ~2,500 pontos ao redor de 10
  Cluster 2: ~2,500 pontos ao redor de 20
  Cluster 3: ~2,500 pontos ao redor de 30

Centróides iniciais escolhidos: [21.93701747 32.87325466  9.31181049 20.97084085]

✓ Arquivos salvos:
  - ../dados/dados_pequeno.csv
  - ../dados/centroides_pequeno.csv

Gerando conjunto: MEDIO
N = 100,000 pontos
K = 8 clusters
Centros reais dos clusters: [0, 10, 20, 30, 40, 50, 60, 70]
Desvio padrão: 2.5
  Cluster 0: ~12,500 pontos ao redor de 0
  Cluster 1: ~12,500 pontos ao redor de 10
  Cluster 2: ~12,500 pontos ao redor de 20
  Cluster 3: ~12,500 pontos ao redor de 30
  Cluster 4: ~12,500 pontos ao redor de 40
  Cluster 5: ~12,500 pontos ao redor de 50
  Cluster 6: ~12,500 pontos ao redor de 60
  Cluster 7: ~12,500 pontos ao redor de 70

Centróides iniciais escolhidos: [39.47531539 69.5562491  2